In [1]:
import numpy as np 
import pandas as pd 
import gc
pd.options.mode.chained_assignment = None  # default='warn'
df_cal = pd.read_csv('calendar.csv')
df_eval = pd.read_csv('sales_train_evaluation.csv')
df_price = pd.read_csv('sell_prices.csv')

In [2]:
hol_list = df_cal['event_name_1'].unique()[1:]
wkd_list = ['Saturday','Sunday']
df_cal['is_holiday_1'] = df_cal['event_name_1'].apply(lambda x : 1 if x in hol_list else 0 )
df_cal['is_holiday_2'] = df_cal['event_name_1'].apply(lambda x : 1 if x in hol_list else 0 )
df_cal['is_holiday'] = df_cal[['is_holiday_1','is_holiday_2']].max(axis=1)
df_cal['is_weekend'] = df_cal['weekday'].apply(lambda x : 1 if x in wkd_list else 0 )
df_cal = df_cal.drop(['weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis='columns')

In [3]:
eva_all = df_eval[df_eval['dept_id'] == 'FOODS_1'] # choose products 
eva_all = eva_all.melt(['id','item_id','dept_id','cat_id','store_id','state_id'],var_name = 'd', value_name = 'sales')
eva_all = pd.merge(eva_all, df_cal, how = 'left', on = 'd')
eva_all = pd.merge(eva_all, df_price, how = 'left', on = ['item_id', 'wm_yr_wk', 'store_id'])
eva_all = eva_all.drop(['is_holiday_1','is_holiday_2'], axis =1)
eva_all

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,0,0,0,0,1,2.00
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,7.88
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,2.88
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,NaN
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,FOODS_1_215_WI_3_evaluation,FOODS_1_215,FOODS_1,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,0,0,0,0,1,1.84
4192556,FOODS_1_216_WI_3_evaluation,FOODS_1_216,FOODS_1,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,0,0,0,0,1,5.28
4192557,FOODS_1_217_WI_3_evaluation,FOODS_1_217,FOODS_1,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,0,0,0,0,1,3.54
4192558,FOODS_1_218_WI_3_evaluation,FOODS_1_218,FOODS_1,FOODS,WI_3,WI,d_1941,20,2016-05-22,11617,0,0,0,0,1,0.98


In [4]:
cat_encode = {"HOBBIES":1,"HOUSEHOLD":2,"FOODS":3}
state_encode = {"CA":1,"TX":2,"WI":3}
dept_encode = {'HOBBIES_1':11, 'HOBBIES_2':12, 
               'HOUSEHOLD_1':21, 'HOUSEHOLD_2':22, 
               'FOODS_1':31,'FOODS_2':32, 'FOODS_3':33}
store_encode = {'CA_1':11, 'CA_2':12, 'CA_3':13, 'CA_4':14, 
                'TX_1':21, 'TX_2':22, 'TX_3':23, 
                'WI_1':31, 'WI_2':32, 'WI_3':33}
def data_encode(df):
    #other_columns = ['cat_id','state_id']
    # use only numerical columns
    df['state_id'] = df['state_id'].replace(state_encode)
    df['cat_id'] = df["cat_id"].replace(cat_encode)
    df['dept_id'] = df["dept_id"].replace(dept_encode)
    df['store_id'] = df["store_id"].replace(store_encode)
    import re
    df['d'] = df['d'].apply(lambda x : int(re.search(r'd_(\d+)', x).group(1)))
    # calendar [["wm_yr_wk", "wday", "month", "year",'d','snap_CA','snap_TX',"snap_WI"]]
    return df
def trans_str(ori_str):
    trans_str = ori_str.replace('_', '')
    trans_str = trans_str.replace('HOBBIES', '1')
    trans_str = trans_str.replace('HOUSEHOLD', '2')
    trans_str = trans_str.replace('FOODS', '3')
    trans_str = trans_str.replace('CA', '1')
    trans_str = trans_str.replace('TX', '2')
    trans_str = trans_str.replace('WI', '3')
    trans_str = trans_str.replace('evaluation', '')
    return trans_str
eva_all_encode = data_encode(eva_all)
eva_all_encode['item_id'] = eva_all_encode['item_id'].apply(trans_str)
eva_all_encode['item_id'] = eva_all_encode['item_id'].astype(int)
eva_all_encode['id'] = eva_all_encode['id'].apply(trans_str)
eva_all_encode['id'] = eva_all_encode['id'].astype(int)
uni_val = eva_all_encode['id'].unique()
mapping = {val: idx for idx, val in enumerate(uni_val)}
eva_all_encode['id'] = np.vectorize(mapping.get)(eva_all_encode['id'])
eva_all_encode = eva_all_encode.drop(['date','wm_yr_wk'],axis=1)#,'id'
eva_all_encode

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,0,31001,31,3,11,1,1,3,0,0,0,0,1,2.00
1,1,31002,31,3,11,1,1,0,0,0,0,0,1,7.88
2,2,31003,31,3,11,1,1,0,0,0,0,0,1,2.88
3,3,31004,31,3,11,1,1,0,0,0,0,0,1,NaN
4,4,31005,31,3,11,1,1,3,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,2155,31215,31,3,33,3,1941,0,0,0,0,0,1,1.84
4192556,2156,31216,31,3,33,3,1941,2,0,0,0,0,1,5.28
4192557,2157,31217,31,3,33,3,1941,1,0,0,0,0,1,3.54
4192558,2158,31218,31,3,33,3,1941,20,0,0,0,0,1,0.98


In [5]:
eva_f1 = eva_all_encode#.drop('id',axis=1)
del eva_all_encode
gc.collect()
eva_f1

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,0,31001,31,3,11,1,1,3,0,0,0,0,1,2.00
1,1,31002,31,3,11,1,1,0,0,0,0,0,1,7.88
2,2,31003,31,3,11,1,1,0,0,0,0,0,1,2.88
3,3,31004,31,3,11,1,1,0,0,0,0,0,1,NaN
4,4,31005,31,3,11,1,1,3,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,2155,31215,31,3,33,3,1941,0,0,0,0,0,1,1.84
4192556,2156,31216,31,3,33,3,1941,2,0,0,0,0,1,5.28
4192557,2157,31217,31,3,33,3,1941,1,0,0,0,0,1,3.54
4192558,2158,31218,31,3,33,3,1941,20,0,0,0,0,1,0.98


In [6]:
def levelG(df,col_1, col_2, level_n, columns):
    df_list = []
    col = []
    for i in [0,1,2,3]:
        col = list(set(col_1[0]) - set(col_1[i]))
        col.append('d')
        sum_L = df.groupby(col)[col_1[i]].sum().reset_index()
        mea_L = df.groupby(col)[col_2[i]].mean().reset_index().iloc[:,i+1:]
        level = pd.concat([sum_L,mea_L],axis=1)[columns]
        df_list.append(level)
    return df_list

In [7]:
# Level data generation:
col_1_L1 = ['sell_price','sales', 'item_id','store_id', 'state_id'] #,'dept_id', 'cat_id' 'id',
col_1_L2 = ['sell_price','sales', 'item_id','store_id'] #,'dept_id', 'cat_id', 'state_id' 'id',
col_1_L3 = ['sell_price','sales', 'item_id'] #,'dept_id', 'cat_id', 'store_id'            'id', 
col_1_L4 = ['sell_price','sales', ] #'id',

col_2_L1 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L2 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L3 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L4 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']

col_1_L = [col_1_L1, 
           col_1_L2,
           col_1_L3,
           col_1_L4]
col_2_L = [col_2_L1, 
           col_2_L2,
           col_2_L3,
           col_2_L4]
columns_name = ['d','item_id', 'store_id', 'state_id', 'snap_CA', 'snap_TX', 'snap_WI', 'is_holiday','is_weekend', 'sell_price','sales']

In [8]:
# generate all levels data
eva_list = levelG(eva_f1, col_1_L, col_2_L, 4, columns_name)

In [9]:
# construct Embedded Matrix 
def collect_xy(df,colname_1,colname_2,x_window:int = 60, y_window: int = 28):
    # for generating x
    df_ = df.copy()
    for i in range(x_window):
        df_[f'x_{i+1}DaysAgo'] = df_[colname_1].shift(i+1)
        
    # for generating y
    for i in range(y_window):
        df_[f"y_{i+1}DaysF"] = df_[colname_2].shift(-i-1)
    df_ = df_.iloc[x_window : -y_window,:]
    return df_

def combine_df_xy(df,group_n, col_list):
    from tqdm import tqdm
    uni_id = df[col_list[0]].unique()
    df_c = pd.DataFrame()
    if group_n == 1:
        for i in tqdm(range(len(uni_id))):
            df_e = df[df[col_list[0]]==uni_id[i]]
            df_e = collect_xy(df_e ,col_list[-1] ,col_list[-1])
            df_c = pd.concat([df_c,df_e])
    elif group_n == 2 : # This number is the amount of level names required for us to disaggregate the hierarchical structure
        for i in tqdm(range(len(uni_id))):
            uni_id_2 = df[col_list[1]].unique()
            for x in range(len(uni_id_2)):
                df_e = df[(df[col_list[0]]==uni_id[i]) & (df[col_list[1]]==uni_id_2[x])]
                df_e = collect_xy(df_e ,col_list[-1] ,col_list[-1])
                df_c = pd.concat([df_c,df_e])
    return df_c

In [10]:
l1 = collect_xy(eva_list[0],'sales','sales')
l2 = combine_df_xy(eva_list[1],1,['state_id', 'sales'])
l3 = combine_df_xy(eva_list[2],1,['store_id', 'sales'])
l4 = combine_df_xy(eva_list[3],2,['store_id','item_id','sales'])
hts = [l1, l2, l3, l4]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [14:47<00:00, 88.75s/it]


In [11]:
# generate the train&test sets
def dataSplit(df):
    x_train, y_train = df[ df['d']<1913 ].iloc[:,:-28], df[df['d']<1913].iloc[:,-28:]
    x_test, y_test   = df[df['d'] >= 1913].iloc[:,:-28], df[df['d'] >= 1913].iloc[:,-28:]
    return x_train,x_test,y_train,y_test

def Concat(df1,df2):
    df = pd.concat([df1,df2], axis=0)
    return df

In [13]:
x1,x2,y1,y2 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()#x_train, x_test, y_train, y_test
for i in range(len(hts)) :
    a, b, c, d = dataSplit(hts[i])
    x1, x2, y1, y2 = Concat(x1, a), Concat(x2, b), Concat(y1, c), Concat(y2, d)
del a,b,c,d
gc.collect()
x_train, x_test, y_train, y_test= x1.iloc[:,4:],x2.iloc[:,4:],y1,y2 # x_train or x_test we could change the number of features

In [14]:
dataset_foods1 = [ x_train, x_test, y_train, y_test] 
file_name = ['x_train','x_test', 'y_train', 'y_test']

In [18]:
# collect dataset for each steps
for i in range(len(hts)):
    hts[i].to_pickle(f"hts_{i}.pkl")

for i in range(len(eva_list)):
    eva_list[i].to_pickle(f"eva_l{i}.pkl")

for i in range(len(dataset_foods1)):
    dataset_foods1[i].to_pickle(f"{file_name[i]}.pkl")
    